# Image Hybrid Neuroevolution Notebook

This notebook implements a hybrid neuroevolution process for image classification (Parkinson detection from spectrograms). The system combines genetic algorithms with 2D convolutional neural networks to evolve optimal architectures for image processing.

## Main Features:
- **Hybrid genetic algorithm**: Combines architecture and weight evolution
- **2D Convolutional Networks**: Optimized for spectrogram image processing
- **5-Fold Cross-Validation**: Each individual is evaluated on all 5 folds sequentially (fitness = average accuracy)
- **CUDA-safe training**: Sequential fold training to avoid GPU memory conflicts
- **Adaptive mutation**: Dynamic mutation rate based on population diversity
- **Image dataset support**: Loads spectrogram images from PNG files
- **Intelligent stopping criteria**: By target fitness or maximum generations
- **Complete visualization**: Shows progress and final best architecture

## Objectives:
1. Create initial population of 2D CNN architectures
2. Evaluate fitness of each individual using **5-fold CV** (robust evaluation)
3. Select best architectures (elitism)
4. Apply crossover and mutation to create new generation
5. Repeat process until convergence
6. Display the best architecture found for Parkinson classification

**✅ Performance**: Sequential 5-fold CV provides robustness against overfitting and is CUDA-safe.

---
## ✨ CONFIGURACIÓN ACTUAL DEL DATASET ✨

**Dataset configurado**: `images_all_real_syn_n` (Espectrogramas como Imágenes PNG)

Este notebook está configurado para usar el **nuevo dataset de imágenes** generado con `npy_to_spectrograms.py`:
- 🎵 **Espectrogramas como Imágenes**: Archivos PNG generados desde datos .npy  
- 🖼️ **Formato**: Imágenes RGB de espectrogramas de audios
- 📊 **5-Fold Cross-Validation**: Organizado en carpetas train/val/test por fold

**Ventajas de este dataset**:
- Mayor diversidad de datos para entrenamiento
- Procesamiento mediante Conv2D (óptimo para imágenes)
- Visualización directa de los espectrogramas
- Estratificación balanceada entre clases (control/patológico)

**🚀 5-Fold Cross-Validation durante la Evolución**: 
- **CADA** individuo se evalúa en **TODOS** los 5 folds **SECUENCIALMENTE**
- Los folds se entrenan uno tras otro para evitar conflictos de CUDA
- El fitness es el **promedio** de accuracy de los 5 folds
- ✅ **CUDA-safe** - evita errores de memoria de GPU
- ✅ **Más robusto** - evita sobreajuste a un fold específico

**📊 Evaluación Final**: 
- Al terminar la evolución, la mejor arquitectura se vuelve a evaluar con 5-fold CV
- Se reportan métricas completas (accuracy, sensitivity, specificity, F1, AUC)

---

## 1. Required Libraries Import

In [1]:
# Install all necessary libraries
import subprocess
import sys

def install_package(package):
    """Installs a package using pip if not available."""
    try:
        __import__(package.split('==')[0].split('[')[0])
        print(f"OK {package.split('==')[0]} is already installed")
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"OK {package} installed correctly")

# List of required packages
required_packages = [
    "torch>=2.0.0",
    "torchvision>=0.15.0",
    "numpy>=1.21.0",
    "matplotlib>=3.5.0",
    "seaborn>=0.11.0",
    "tqdm>=4.64.0",
    "jupyter>=1.0.0",
    "ipywidgets>=8.0.0",
    "Pillow>=9.0.0",  # For image loading
    "scikit-learn>=1.0.0"
]

print("Starting dependency installation for Image Hybrid Neuroevolution...")
print("=" * 60)

for package in required_packages:
    install_package(package)

print("\nAll dependencies have been verified/installed")
print("Restart the kernel if this is the first time installing torch")
print("=" * 60)

# Verify PyTorch installation
try:
    import torch
    print(f"\nPyTorch {torch.__version__} installed correctly")
    print(f"CUDA available: {'Yes' if torch.cuda.is_available() else 'No'}")
    if torch.cuda.is_available():
        print(f"GPU detected: {torch.cuda.get_device_name(0)}")
        print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory // 1024**3} GB")
except ImportError:
    print("ERROR: PyTorch could not be installed correctly")
    print("Try installing manually with: pip install torch torchvision")

Starting dependency installation for Image Hybrid Neuroevolution...
Installing torch>=2.0.0...


OK torch>=2.0.0 installed correctly
Installing torchvision>=0.15.0...


OK torchvision>=0.15.0 installed correctly
Installing numpy>=1.21.0...


OK numpy>=1.21.0 installed correctly
Installing matplotlib>=3.5.0...


OK matplotlib>=3.5.0 installed correctly
Installing seaborn>=0.11.0...


...

OK tqdm>=4.64.0 installed correctly
Installing jupyter>=1.0.0...


OK jupyter>=1.0.0 installed correctly
Installing ipywidgets>=8.0.0...


OK ipywidgets>=8.0.0 installed correctly
Installing Pillow>=9.0.0...


OK Pillow>=9.0.0 installed correctly
Installing scikit-learn>=1.0.0...


OK scikit-learn>=1.0.0 installed correctly

All dependencies have been verified/installed
Restart the kernel if this is the first time installing torch

PyTorch 2.8.0+cu128 installed correctly
CUDA available: No


In [2]:
# Main imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image

# Scientific libraries
import numpy as np
import random
import copy
import json
import os
from pathlib import Path
from typing import Dict, List, Tuple, Any
from datetime import datetime
import uuid
import glob

# Threading for parallel fold training
import threading
from queue import Queue
from concurrent.futures import ThreadPoolExecutor, as_completed

# Visualization and progress
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# Configure logging
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Configure seeds for reproducibility
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)

# Configure device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device configured: {device}")
print(f"PyTorch version: {torch.__version__}")

# Suppress unnecessary warnings
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

Device configured: cpu
PyTorch version: 2.8.0+cu128


## 2. System Configuration and Parameters

In [3]:
# Main genetic algorithm configuration
CONFIG = {
    # Genetic algorithm parameters
    'population_size': 20,
    'max_generations': 100,
    'fitness_threshold': 80.0,

    # Adaptive mutation parameters
    'base_mutation_rate': 0.25,
    'mutation_rate_min': 0.10,
    'mutation_rate_max': 0.80,
    'current_mutation_rate': 0.25,

    'crossover_rate': 0.99,
    'elite_percentage': 0.2,

    # Dataset selection (IMAGE)
    'dataset': 'IMAGE',

    # Dataset parameters for images (spectrograms)
    'num_channels': 3,              # RGB images
    'image_height': 32,             # Fixed size for spectrograms
    'image_width': 64,              # Fixed size for spectrograms
    'num_classes': 2,               # Control vs Pathological
    'batch_size': 64,               # Smaller batch for images

    # Training parameters
    'num_epochs': 10,
    'learning_rate': 0.0001,
    'early_stopping_patience': 40, #256+50 8k images (all dataset)

    # Epoch-level early stopping
    'epoch_patience': 2,
    'improvement_threshold': 0.01,

    # Generation-level early stopping
    'early_stopping_generations': 20,
    'min_improvement_threshold': 0.01,

    # Allowed architecture range for 2D Conv
    'min_conv_layers': 1,
    'max_conv_layers': 8,           # Fewer layers for images
    'min_fc_layers': 1,
    'max_fc_layers': 8,
    'min_filters': 8,
    'max_filters': 128,
    'min_fc_nodes': 64,
    'max_fc_nodes': 1024,

    # Mutation parameters - Kernel sizes for 2D Conv
    'kernel_size_options': [1, 3, 5, 7],  # Common for 2D convolutions
    
    # Mutation parameters - Dropout range
    'min_dropout': 0.2,
    'max_dropout': 0.5,
    
    # Mutation parameters - Learning rate options
    'learning_rate_options': [0.001, 0.0005, 0.0001, 0.00005, 0.00001],
    
    # Mutation parameters - Normalization type weights
    'normalization_batch_weight': 0.9,
    'normalization_layer_weight': 0.1,

    # Image dataset configuration
    'dataset_id': 'all_real_syn_n',
    'fold_id': 'all_real_syn_n',
    'num_folds': 5,
    'data_path': os.path.join('data', 'sets', 'folds_5', 'images_all_real_syn_n'),
    
    # Image normalization (ImageNet stats)
    'normalization': {
        'mean': [0.485, 0.456, 0.406],
        'std': [0.229, 0.224, 0.225]
    }
}

# Activation function mapping
ACTIVATION_FUNCTIONS = {
    'relu': nn.ReLU,
    'leaky_relu': nn.LeakyReLU,
    'tanh': nn.Tanh,
    'sigmoid': nn.Sigmoid,
    'selu': nn.SELU,
}

# Optimizer mapping
OPTIMIZERS = {
    'adam': optim.Adam,
    'adamw': optim.AdamW,
    'sgd': optim.SGD,
    'rmsprop': optim.RMSprop,
}

print("Configuration loaded (adaptive mutation enabled, 2D Conv for images):")
print(f"   Dataset: Images (Spectrogram - Parkinson Classification)")
for key, value in CONFIG.items():
    if key not in ['normalization']:
        print(f"   {key}: {value}")
print(f"\nAvailable activation functions: {list(ACTIVATION_FUNCTIONS.keys())}")
print(f"Available optimizers: {list(OPTIMIZERS.keys())}")
print(f"\nData path configured: {CONFIG['data_path']}")

Configuration loaded (adaptive mutation enabled, 2D Conv for images):
   Dataset: Images (Spectrogram - Parkinson Classification)
   population_size: 20
   max_generations: 100
   fitness_threshold: 80.0
   base_mutation_rate: 0.25
   mutation_rate_min: 0.1
   mutation_rate_max: 0.8
   current_mutation_rate: 0.25
   crossover_rate: 0.99
   elite_percentage: 0.2
   dataset: IMAGE
   num_channels: 3
   image_height: 32
   image_width: 64
   num_classes: 2
   batch_size: 64
   num_epochs: 10
   learning_rate: 0.0001
   early_stopping_patience: 40
   epoch_patience: 2
   improvement_threshold: 0.01
   early_stopping_generations: 20
   min_improvement_threshold: 0.01
   min_conv_layers: 1
   max_conv_layers: 8
   min_fc_layers: 1
   max_fc_layers: 8
   min_filters: 8
   max_filters: 128
   min_fc_nodes: 64
   max_fc_nodes: 1024
   kernel_size_options: [1, 3, 5, 7]
   min_dropout: 0.2
   max_dropout: 0.5
   learning_rate_options: [0.001, 0.0005, 0.0001, 5e-05, 1e-05]
   normalization_batch_w

## 3. Dataset Loading and Preprocessing

In [4]:
class SpectrogramDataset(Dataset):
    """Dataset class for loading spectrogram images."""
    
    def __init__(self, image_paths: List[str], labels: List[int], transform=None):
        """
        Args:
            image_paths: List of paths to image files
            labels: List of labels (0=control, 1=pathological)
            transform: Optional transform to apply to images
        """
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        # Load image
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        
        # Apply transforms
        if self.transform:
            image = self.transform(image)
        
        label = self.labels[idx]
        
        return image, label


def load_fold_images(data_path: str, fold_num: int, subset: str) -> Tuple[List[str], List[int]]:
    """
    Load image paths and labels for a specific fold and subset.
    
    Args:
        data_path: Base path to image directory
        fold_num: Fold number (1-5)
        subset: 'train', 'val', or 'test'
    
    Returns:
        Tuple of (image_paths, labels)
    """
    subset_folder = os.path.join(data_path, f"{subset}_fold_{fold_num}")
    
    if not os.path.exists(subset_folder):
        raise FileNotFoundError(f"Folder not found: {subset_folder}")
    
    # Get all image files
    image_paths = []
    labels = []
    
    # Scan for all .png files
    for img_file in sorted(glob.glob(os.path.join(subset_folder, "*.png"))):
        filename = os.path.basename(img_file)
        
        # Extract label from filename (control=0, pathological=1)
        if 'control' in filename.lower():
            label = 0
        elif 'pathological' in filename.lower():
            label = 1
        else:
            logger.warning(f"Unknown label for file: {filename}, skipping...")
            continue
        
        image_paths.append(img_file)
        labels.append(label)
    
    return image_paths, labels


def verify_dataset(config: dict):
    """
    Verifies that image data exists and loads first image to detect dimensions.
    
    Args:
        config: Configuration dictionary
    """
    print("\n" + "="*60)
    print("VERIFICANDO DISPONIBILIDAD DE IMÁGENES")
    print("="*60)
    print(f"Dataset: {config['dataset_id']}, Verificando los 5 folds...")
    
    data_path = config['data_path']
    print(f"   Looking for: {os.path.abspath(data_path)}")
    
    # Check if directory exists
    if not os.path.exists(data_path):
        raise FileNotFoundError(
            f"\n❌ Data directory not found!\n"
            f"   Expected: {os.path.abspath(data_path)}\n"
            f"   Please run npy_to_spectrograms.py first to generate images."
        )
    
    print(f"   ✓ Directory found: {os.path.abspath(data_path)}")
    
    # Check that all 5 folds exist
    print(f"\nChecking for all 5 folds...")
    all_folds_ok = True
    total_images = 0
    
    for fold_num in range(1, 6):
        fold_ok = True
        fold_images = 0
        
        for subset in ['train', 'val', 'test']:
            subset_folder = os.path.join(data_path, f"{subset}_fold_{fold_num}")
            
            if not os.path.exists(subset_folder):
                fold_ok = False
                all_folds_ok = False
                print(f"   ✗ Fold {fold_num}: Missing {subset} folder")
                break
            
            # Count images
            num_images = len(glob.glob(os.path.join(subset_folder, "*.png")))
            fold_images += num_images
        
        if fold_ok:
            print(f"   ✓ Fold {fold_num}: {fold_images} images")
            total_images += fold_images
    
    if not all_folds_ok:
        raise FileNotFoundError(
            f"\n❌ Some fold folders are missing!\n"
            f"   Please run npy_to_spectrograms.py to generate all fold images.\n"
        )
    
    print(f"\n✓ All 5 folds verified successfully!")
    print(f"   Total images: {total_images}")
    
    # Load first image to detect dimensions
    print(f"\nDetecting original image dimensions...")
    image_paths, _ = load_fold_images(data_path, 1, 'train')
    
    if len(image_paths) == 0:
        raise FileNotFoundError("No images found in train_fold_1!")
    
    first_image = Image.open(image_paths[0]).convert('RGB')
    original_width, original_height = first_image.size
    
    # DO NOT overwrite config - keep the target resize dimensions
    target_width = config['image_width']
    target_height = config['image_height']
    
    print(f"   Original image dimensions: {original_width} x {original_height}")
    print(f"   Target resize dimensions: {target_width} x {target_height}")
    print(f"   Channels: {config['num_channels']} (RGB)")
    print(f"\n✓ Dataset verification complete!")
    print(f"   During evolution, each individual will train on all 5 folds.")
    print("="*60)


# Verify dataset availability
verify_dataset(CONFIG)

print(f"\n{'='*60}")
print("DATASET READY FOR 5-FOLD CROSS-VALIDATION")
print(f"{'='*60}")
print(f"   Image dimensions: {CONFIG['image_width']} x {CONFIG['image_height']}")
print(f"   Input channels: {CONFIG['num_channels']}")
print(f"   Number of classes: {CONFIG['num_classes']}")
print(f"   Batch size: {CONFIG['batch_size']}")
print(f"   Task: Spectrogram classification - Control (0) vs Pathological (1)")
print(f"\n   ⚠️ Each individual will be evaluated on ALL 5 folds SEQUENTIALLY")
print(f"   ⚠️ This is slower but CUDA-safe (avoids GPU memory conflicts)")


VERIFICANDO DISPONIBILIDAD DE IMÁGENES
Dataset: all_real_syn_n, Verificando los 5 folds...
   Looking for: /home/jovyan/audio_test/data/sets/folds_5/images_all_real_syn_n
   ✓ Directory found: /home/jovyan/audio_test/data/sets/folds_5/images_all_real_syn_n

Checking for all 5 folds...
   ✓ Fold 1: 12220 images
   ✓ Fold 2: 12180 images
   ✓ Fold 3: 12200 images
   ✓ Fold 4: 12220 images
   ✓ Fold 5: 12240 images

✓ All 5 folds verified successfully!
   Total images: 61060

Detecting original image dimensions...
   Original image dimensions: 1681 x 1181
   Target resize dimensions: 64 x 32
   Channels: 3 (RGB)

✓ Dataset verification complete!
   During evolution, each individual will train on all 5 folds.

DATASET READY FOR 5-FOLD CROSS-VALIDATION
   Image dimensions: 64 x 32
   Input channels: 3
   Number of classes: 2
   Batch size: 64
   Task: Spectrogram classification - Control (0) vs Pathological (1)

   ⚠️ Each individual will be evaluated on ALL 5 folds SEQUENTIALLY
   ⚠️ This

## 4. Neural Network Architecture Definition (2D CNN)

In [5]:
class EvolvableCNN2D(nn.Module):
    """
    Evolvable CNN class for 2D image processing.
    Uses Conv2D layers for spectrogram/image data.
    """
    
    def __init__(self, genome: dict, config: dict):
        super(EvolvableCNN2D, self).__init__()
        self.genome = genome
        self.config = config
        
        # Validate and fix genome structure
        self._validate_genome()
        
        # Build convolutional layers (2D for images)
        self.conv_layers = self._build_conv_layers()
        
        # Calculate output size after convolutions
        self.conv_output_size = self._calculate_conv_output_size()
        
        # Build fully connected layers
        self.fc_layers = self._build_fc_layers()
    
    def _validate_genome(self):
        """Validates and fixes genome structure to ensure consistency."""
        num_conv = self.genome['num_conv_layers']
        
        # Fix filters list
        if len(self.genome['filters']) != num_conv:
            self.genome['filters'] = self.genome['filters'][:num_conv]
            while len(self.genome['filters']) < num_conv:
                self.genome['filters'].append(
                    random.randint(self.config['min_filters'], self.config['max_filters'])
                )
        
        # Fix kernel_sizes list
        if len(self.genome['kernel_sizes']) != num_conv:
            self.genome['kernel_sizes'] = self.genome['kernel_sizes'][:num_conv]
            while len(self.genome['kernel_sizes']) < num_conv:
                self.genome['kernel_sizes'].append(
                    random.choice(self.config['kernel_size_options'])
                )
        
        # Fix use_pooling list
        if len(self.genome['use_pooling']) != num_conv:
            self.genome['use_pooling'] = self.genome['use_pooling'][:num_conv]
            while len(self.genome['use_pooling']) < num_conv:
                self.genome['use_pooling'].append(random.choice([True, False]))
        
        # Fix use_batch_norm list
        if len(self.genome['use_batch_norm']) != num_conv:
            self.genome['use_batch_norm'] = self.genome['use_batch_norm'][:num_conv]
            while len(self.genome['use_batch_norm']) < num_conv:
                self.genome['use_batch_norm'].append(random.choice([True, False]))
        
        # Fix fc_nodes list
        num_fc = self.genome['num_fc_layers']
        if len(self.genome['fc_nodes']) != num_fc:
            self.genome['fc_nodes'] = self.genome['fc_nodes'][:num_fc]
            while len(self.genome['fc_nodes']) < num_fc:
                self.genome['fc_nodes'].append(
                    random.randint(self.config['min_fc_nodes'], self.config['max_fc_nodes'])
                )
    
    def _build_conv_layers(self):
        """Build the convolutional layers based on genome (2D)."""
        layers = nn.ModuleList()
        
        in_channels = self.config['num_channels']
        
        for i in range(self.genome['num_conv_layers']):
            out_channels = self.genome['filters'][i]
            kernel_size = self.genome['kernel_sizes'][i]
            
            # Conv2D layer
            conv = nn.Conv2d(
                in_channels=in_channels,
                out_channels=out_channels,
                kernel_size=kernel_size,
                padding=kernel_size // 2  # Same padding
            )
            layers.append(conv)
            
            # Batch normalization (2D)
            if self.genome['use_batch_norm'][i]:
                layers.append(nn.BatchNorm2d(out_channels))
            
            # Activation
            activation_class = ACTIVATION_FUNCTIONS[self.genome['activation']]
            layers.append(activation_class())
            
            # Pooling (2D)
            if self.genome['use_pooling'][i]:
                layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
            
            in_channels = out_channels
        
        return layers
    
    def _calculate_conv_output_size(self):
        """Calculate the output size after all conv layers."""
        # Create a dummy input
        dummy_input = torch.zeros(
            1, 
            self.config['num_channels'],
            self.config['image_height'],
            self.config['image_width']
        )
        
        # Pass through conv layers
        x = dummy_input
        for layer in self.conv_layers:
            x = layer(x)
        
        # Flatten and return size
        return x.view(1, -1).size(1)
    
    def _build_fc_layers(self):
        """Build the fully connected layers."""
        layers = nn.ModuleList()
        
        in_features = self.conv_output_size
        
        for i in range(self.genome['num_fc_layers']):
            out_features = self.genome['fc_nodes'][i]
            
            # Fully connected layer
            fc = nn.Linear(in_features, out_features)
            layers.append(fc)
            
            # Activation
            activation_class = ACTIVATION_FUNCTIONS[self.genome['activation']]
            layers.append(activation_class())
            
            # Dropout
            layers.append(nn.Dropout(self.genome['dropout_rate']))
            
            in_features = out_features
        
        # Output layer
        layers.append(nn.Linear(in_features, self.config['num_classes']))
        
        return layers
    
    def forward(self, x):
        """Forward pass through the network."""
        # Convolutional layers
        for layer in self.conv_layers:
            x = layer(x)
        
        # Flatten
        x = x.view(x.size(0), -1)
        
        # Fully connected layers
        for layer in self.fc_layers:
            x = layer(x)
        
        return x


# Test the architecture
test_genome = {
    'num_conv_layers': 3,
    'filters': [32, 64, 128],
    'kernel_sizes': [3, 3, 3],
    'use_pooling': [True, True, False],
    'use_batch_norm': [True, True, True],
    'num_fc_layers': 2,
    'fc_nodes': [256, 128],
    'activation': 'relu',
    'dropout_rate': 0.3,
    'optimizer': 'adam',
    'learning_rate': 0.0001
}

test_model = EvolvableCNN2D(test_genome, CONFIG)
print(f"\nTest model created successfully!")
print(f"Conv output size: {test_model.conv_output_size}")
print(f"Total parameters: {sum(p.numel() for p in test_model.parameters()):,}")

# Test forward pass
test_input = torch.randn(2, CONFIG['num_channels'], CONFIG['image_height'], CONFIG['image_width'])
test_output = test_model(test_input)
print(f"Output shape: {test_output.shape}")
print(f"Expected shape: torch.Size([2, {CONFIG['num_classes']}])")


Test model created successfully!
Conv output size: 16384
Total parameters: 4,321,410
Output shape: torch.Size([2, 2])
Expected shape: torch.Size([2, 2])


## 5. Genetic Algorithm Components

In [6]:
def create_random_genome(config: dict) -> dict:
    """Creates a random genome for 2D CNN architectures."""
    num_conv_layers = random.randint(config['min_conv_layers'], config['max_conv_layers'])
    num_fc_layers = random.randint(config['min_fc_layers'], config['max_fc_layers'])

    # Filters (progressive increase)
    filters = []
    base_filters = random.randint(config['min_filters'], config['min_filters'] * 2)
    for i in range(num_conv_layers):
        layer_filters = min(base_filters * (2 ** i), config['max_filters'])
        filters.append(layer_filters)

    # Kernel sizes for 2D
    kernel_sizes = [random.choice(config['kernel_size_options']) for _ in range(num_conv_layers)]

    # Pooling and batch norm decisions
    use_pooling = [random.choice([True, False]) for _ in range(num_conv_layers)]
    use_batch_norm = [random.choice([True, False]) for _ in range(num_conv_layers)]

    # FC nodes (progressive decrease)
    fc_nodes = []
    base_fc = random.randint(config['min_fc_nodes'], config['max_fc_nodes'])
    for i in range(num_fc_layers):
        layer_nodes = max(config['min_fc_nodes'], base_fc // (2 ** i))
        fc_nodes.append(layer_nodes)

    # Activation function (single for all layers)
    activation = random.choice(list(ACTIVATION_FUNCTIONS.keys()))

    # Other parameters
    dropout_rate = random.uniform(config['min_dropout'], config['max_dropout'])
    learning_rate = random.choice(config['learning_rate_options'])
    optimizer = random.choice(list(OPTIMIZERS.keys()))

    genome = {
        'num_conv_layers': num_conv_layers,
        'num_fc_layers': num_fc_layers,
        'filters': filters,
        'kernel_sizes': kernel_sizes,
        'use_pooling': use_pooling,
        'use_batch_norm': use_batch_norm,
        'fc_nodes': fc_nodes,
        'activation': activation,
        'dropout_rate': dropout_rate,
        'learning_rate': learning_rate,
        'optimizer': optimizer,
        'fitness': 0.0,
        'id': str(uuid.uuid4())[:8]
    }
    return genome


def mutate_genome(genome: dict, config: dict) -> dict:
    """Mutates a genome for 2D CNN."""
    mutated = copy.deepcopy(genome)
    mutation_rate = config['current_mutation_rate']

    # Mutate layer counts
    if random.random() < mutation_rate:
        mutated['num_conv_layers'] = random.randint(config['min_conv_layers'], config['max_conv_layers'])
    if random.random() < mutation_rate:
        mutated['num_fc_layers'] = random.randint(config['min_fc_layers'], config['max_fc_layers'])

    # Adjust lists to match new sizes
    while len(mutated['filters']) < mutated['num_conv_layers']:
        mutated['filters'].append(random.randint(config['min_filters'], config['max_filters']))
    mutated['filters'] = mutated['filters'][:mutated['num_conv_layers']]

    while len(mutated['kernel_sizes']) < mutated['num_conv_layers']:
        mutated['kernel_sizes'].append(random.choice(config['kernel_size_options']))
    mutated['kernel_sizes'] = mutated['kernel_sizes'][:mutated['num_conv_layers']]

    while len(mutated['use_pooling']) < mutated['num_conv_layers']:
        mutated['use_pooling'].append(random.choice([True, False]))
    mutated['use_pooling'] = mutated['use_pooling'][:mutated['num_conv_layers']]

    while len(mutated['use_batch_norm']) < mutated['num_conv_layers']:
        mutated['use_batch_norm'].append(random.choice([True, False]))
    mutated['use_batch_norm'] = mutated['use_batch_norm'][:mutated['num_conv_layers']]

    while len(mutated['fc_nodes']) < mutated['num_fc_layers']:
        mutated['fc_nodes'].append(random.randint(config['min_fc_nodes'], config['max_fc_nodes']))
    mutated['fc_nodes'] = mutated['fc_nodes'][:mutated['num_fc_layers']]

    # Mutate individual values
    for i in range(len(mutated['filters'])):
        if random.random() < mutation_rate:
            mutated['filters'][i] = random.randint(config['min_filters'], config['max_filters'])

    for i in range(len(mutated['kernel_sizes'])):
        if random.random() < mutation_rate:
            mutated['kernel_sizes'][i] = random.choice(config['kernel_size_options'])

    for i in range(len(mutated['fc_nodes'])):
        if random.random() < mutation_rate:
            mutated['fc_nodes'][i] = random.randint(config['min_fc_nodes'], config['max_fc_nodes'])

    if random.random() < mutation_rate:
        mutated['activation'] = random.choice(list(ACTIVATION_FUNCTIONS.keys()))
    if random.random() < mutation_rate:
        mutated['dropout_rate'] = random.uniform(config['min_dropout'], config['max_dropout'])
    if random.random() < mutation_rate:
        mutated['learning_rate'] = random.choice(config['learning_rate_options'])
    if random.random() < mutation_rate:
        mutated['optimizer'] = random.choice(list(OPTIMIZERS.keys()))

    mutated['id'] = str(uuid.uuid4())[:8]
    mutated['fitness'] = 0.0
    return mutated


def crossover_genomes(parent1: dict, parent2: dict, config: dict) -> Tuple[dict, dict]:
    """Crossover between two genomes."""
    if random.random() > config['crossover_rate']:
        return copy.deepcopy(parent1), copy.deepcopy(parent2)

    child1 = copy.deepcopy(parent1)
    child2 = copy.deepcopy(parent2)

    # Crossover scalar parameters
    for key in ['num_conv_layers', 'num_fc_layers', 'dropout_rate', 'learning_rate', 'optimizer', 'activation']:
        if random.random() < 0.5:
            child1[key], child2[key] = child2[key], child1[key]

    # Crossover lists
    for list_key in ['filters', 'kernel_sizes', 'use_pooling', 'use_batch_norm', 'fc_nodes']:
        if random.random() < 0.5 and len(child1[list_key]) > 1 and len(child2[list_key]) > 1:
            point1 = random.randint(1, len(child1[list_key]) - 1)
            point2 = random.randint(1, len(child2[list_key]) - 1)
            child1[list_key] = child1[list_key][:point1] + child2[list_key][point2:]
            child2[list_key] = child2[list_key][:point2] + child1[list_key][point1:]

    child1['id'] = str(uuid.uuid4())[:8]
    child2['id'] = str(uuid.uuid4())[:8]
    child1['fitness'] = 0.0
    child2['fitness'] = 0.0

    # Fix sizes
    for child in [child1, child2]:
        while len(child['filters']) < child['num_conv_layers']:
            child['filters'].append(random.randint(config['min_filters'], config['max_filters']))
        child['filters'] = child['filters'][:child['num_conv_layers']]
        
        while len(child['kernel_sizes']) < child['num_conv_layers']:
            child['kernel_sizes'].append(random.choice(config['kernel_size_options']))
        child['kernel_sizes'] = child['kernel_sizes'][:child['num_conv_layers']]
        
        while len(child['use_pooling']) < child['num_conv_layers']:
            child['use_pooling'].append(random.choice([True, False]))
        child['use_pooling'] = child['use_pooling'][:child['num_conv_layers']]
        
        while len(child['use_batch_norm']) < child['num_conv_layers']:
            child['use_batch_norm'].append(random.choice([True, False]))
        child['use_batch_norm'] = child['use_batch_norm'][:child['num_conv_layers']]
        
        while len(child['fc_nodes']) < child['num_fc_layers']:
            child['fc_nodes'].append(random.randint(config['min_fc_nodes'], config['max_fc_nodes']))
        child['fc_nodes'] = child['fc_nodes'][:child['num_fc_layers']]

    return child1, child2


print("✓ Genetic functions defined for 2D CNN (create, mutate, crossover)")

✓ Genetic functions defined for 2D CNN (create, mutate, crossover)


## 6. Hybrid Neuroevolution Implementation with 5-Fold CV

In [7]:
class HybridNeuroevolutionImage:
    """Hybrid neuroevolution for 2D image classification with 5-fold CV (sequential)."""

    def __init__(self, config: dict):
        self.config = config
        self.population = []
        self.generation = 0
        self.best_individual = None
        self.fitness_history = []
        self.generation_stats = []
        self.generations_without_improvement = 0
        self.best_fitness_overall = -float('inf')

    def initialize_population(self):
        """Initialize random population."""
        print(f"Initializing population of {self.config['population_size']} individuals...")
        self.population = [create_random_genome(self.config) for _ in range(self.config['population_size'])]
        print(f"✓ Population initialized with {len(self.population)} individuals")

    def _load_fold_data(self, fold_num: int) -> Tuple[DataLoader, DataLoader]:
        """Load train and test data for a specific fold."""
        # Image transforms
        print(f"      Loading data for Fold {fold_num}...")
        print(f"         Target resize: {self.config['image_height']}x{self.config['image_width']} (HxW), Channels: {self.config['num_channels']}")
        transform = transforms.Compose([
            transforms.Resize((self.config['image_height'], self.config['image_width'])),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=self.config['normalization']['mean'],
                std=self.config['normalization']['std']
            )
        ])

        # Load image paths and labels
        train_paths, train_labels = load_fold_images(self.config['data_path'], fold_num, 'train')
        test_paths, test_labels = load_fold_images(self.config['data_path'], fold_num, 'test')

        # Create datasets
        train_dataset = SpectrogramDataset(train_paths, train_labels, transform=transform)
        test_dataset = SpectrogramDataset(test_paths, test_labels, transform=transform)

        # Create dataloaders
        train_loader = DataLoader(
            train_dataset,
            batch_size=self.config['batch_size'],
            shuffle=True,
            num_workers=0,
            pin_memory=True
        )

        test_loader = DataLoader(
            test_dataset,
            batch_size=self.config['batch_size'],
            shuffle=False,
            num_workers=0,
            pin_memory=True
        )

        return train_loader, test_loader

    def _train_one_fold(self, model, optimizer, criterion, train_loader, test_loader, genome_id: str, fold_num: int) -> float:
        """Train and evaluate model on one fold."""
        best_acc = 0.0
        patience_left = self.config['epoch_patience']
        last_improvement_acc = 0.0
        max_epochs = self.config['num_epochs']

        for epoch in range(1, max_epochs + 1):
            # Training
            model.train()
            running_loss = 0.0
            batch_count = 0
            max_batches = min(len(train_loader), self.config['early_stopping_patience'])

            for data, target in train_loader:
                data, target = data.to(device, non_blocking=True), target.to(device, non_blocking=True)
                optimizer.zero_grad()
                output = model(data)
                loss = criterion(output, target)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
                batch_count += 1
                if batch_count >= max_batches:
                    break

            avg_loss = running_loss / max(1, batch_count)

            # Evaluation
            model.eval()
            correct = 0
            total = 0
            eval_batches = 0
            max_eval_batches = min(len(test_loader), 20)

            with torch.no_grad():
                for data, target in test_loader:
                    data, target = data.to(device, non_blocking=True), target.to(device, non_blocking=True)
                    output = model(data)
                    _, predicted = torch.max(output, 1)
                    total += target.size(0)
                    correct += (predicted == target).sum().item()
                    eval_batches += 1
                    if eval_batches >= max_eval_batches:
                        break

            acc = 100.0 * correct / max(1, total)

            # Early stopping
            improvement = acc - last_improvement_acc
            if improvement >= self.config['improvement_threshold']:
                patience_left = self.config['epoch_patience']
                last_improvement_acc = acc
            else:
                patience_left -= 1

            if acc > best_acc:
                best_acc = acc

            # Log every 1 epochs (including first)
            if epoch % 1 == 0 or epoch == 1:
                print(f"          Fold {fold_num} Epoch {epoch}/{max_epochs}: loss={avg_loss:.4f}, acc={acc:.2f}%, best={best_acc:.2f}%, patience={patience_left}")

            if patience_left <= 0:
                print(f"          Fold {fold_num}: Early stopping at epoch {epoch}")
                break

        return best_acc

    def _train_fold_in_thread(self, genome: dict, fold_num: int) -> Tuple[int, float]:
        """Train model on a specific fold (designed for threading)."""
        try:
            # Load fold data
            fold_train_loader, fold_test_loader = self._load_fold_data(fold_num)

            # Create model for this fold
            model = EvolvableCNN2D(genome, self.config).to(device)
            optimizer_class = OPTIMIZERS[genome['optimizer']]
            optimizer = optimizer_class(model.parameters(), lr=genome['learning_rate'])
            criterion = nn.CrossEntropyLoss()

            # Train and evaluate
            fold_acc = self._train_one_fold(
                model, optimizer, criterion,
                fold_train_loader, fold_test_loader,
                genome['id'], fold_num
            )

            print(f"      → Fold {fold_num} completed: {fold_acc:.2f}%")

            return fold_num, fold_acc

        except Exception as e:
            print(f"      ERROR in Fold {fold_num}: {e}")
            import traceback
            traceback.print_exc()
            return fold_num, 0.0

    def _display_architecture(self, genome: dict):
        """Display architecture details of an individual."""
        print(f"\n   📐 Architecture of Individual {genome['id']}:")
        print(f"      Conv2D Layers: {genome['num_conv_layers']}")
        for i in range(genome['num_conv_layers']):
            pooling = "Pool" if genome['use_pooling'][i] else "No-Pool"
            bn = "BN" if genome['use_batch_norm'][i] else "No-BN"
            print(f"         L{i+1}: {genome['filters'][i]:3d} filters, K={genome['kernel_sizes'][i]}x{genome['kernel_sizes'][i]}, {pooling}, {bn}")
        
        print(f"      FC Layers: {genome['num_fc_layers']}")
        for i in range(genome['num_fc_layers']):
            print(f"         FC{i+1}: {genome['fc_nodes'][i]:4d} nodes")
        
        print(f"      Activation: {genome['activation']}, Dropout: {genome['dropout_rate']:.3f}")
        print(f"      Optimizer: {genome['optimizer']}, LR: {genome['learning_rate']}")

    def evaluate_individual(self, genome: dict) -> float:
        """Evaluate individual on ALL 5 folds SEQUENTIALLY (CUDA-safe)."""
        print(f"\n   Evaluating Individual {genome['id']}...")
        
        # Display architecture before training
        self._display_architecture(genome)

        # Train all 5 folds sequentially to avoid CUDA conflicts
        fold_accuracies = {}

        for fold_num in range(1, 6):
            # Clear CUDA cache before each fold
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
            
            print(f"      Training Fold {fold_num}/5...")
            fold_num_result, fold_acc = self._train_fold_in_thread(genome, fold_num)
            fold_accuracies[fold_num_result] = fold_acc
            print(f"      → Fold {fold_num} completed: {fold_acc:.2f}%")

        # Calculate average fitness across all 5 folds
        avg_fitness = np.mean(list(fold_accuracies.values()))

        print(f"   → Individual {genome['id']}: Average Fitness = {avg_fitness:.2f}%")
        print(f"      Fold details: {fold_accuracies}")

        return avg_fitness

    def evolve(self) -> dict:
        """Main evolution loop."""
        print("\n" + "="*60)
        print("STARTING HYBRID NEUROEVOLUTION FOR IMAGES")
        print("="*60)

        self.initialize_population()

        for generation in range(1, self.config['max_generations'] + 1):
            self.generation = generation
            print(f"\n{'='*60}")
            print(f"GENERATION {generation}/{self.config['max_generations']}")
            print(f"{'='*60}")

            # Evaluate all individuals
            print(f"\nEvaluating {len(self.population)} individuals (5-fold CV)...")
            for idx, individual in enumerate(self.population, 1):
                print(f"\nIndividual {idx}/{len(self.population)} - ID: {individual['id']}")
                fitness = self.evaluate_individual(individual)
                individual['fitness'] = fitness

            # Sort by fitness
            self.population.sort(key=lambda x: x['fitness'], reverse=True)
            best_current = self.population[0]
            avg_fitness = np.mean([ind['fitness'] for ind in self.population])

            # Update best overall
            if best_current['fitness'] > self.best_fitness_overall:
                improvement = best_current['fitness'] - self.best_fitness_overall
                self.best_fitness_overall = best_current['fitness']
                self.best_individual = copy.deepcopy(best_current)
                self.generations_without_improvement = 0
                print(f"\n🎉 NEW BEST! Fitness: {best_current['fitness']:.2f}% (improvement: +{improvement:.2f}%)")
            else:
                self.generations_without_improvement += 1

            # Stats
            self.fitness_history.append(avg_fitness)
            self.generation_stats.append({
                'generation': generation,
                'best_fitness': best_current['fitness'],
                'avg_fitness': avg_fitness,
                'best_id': best_current['id']
            })

            print(f"\nGeneration {generation} Summary:")
            print(f"   Best: {best_current['fitness']:.2f}% (ID: {best_current['id']})")
            print(f"   Average: {avg_fitness:.2f}%")
            print(f"   Generations without improvement: {self.generations_without_improvement}")

            # Check stopping criteria
            if best_current['fitness'] >= self.config['fitness_threshold']:
                print(f"\n✓ TARGET REACHED! Fitness {best_current['fitness']:.2f}% >= {self.config['fitness_threshold']}%")
                break

            if self.generations_without_improvement >= self.config['early_stopping_generations']:
                print(f"\n⏹ Early stopping: No improvement for {self.generations_without_improvement} generations")
                break

            # Create new generation (elitism + mutation + crossover)
            num_elites = max(1, int(len(self.population) * self.config['elite_percentage']))
            new_population = self.population[:num_elites]  # Keep best individuals

            while len(new_population) < self.config['population_size']:
                # Selection (tournament)
                parent1 = random.choice(self.population[:len(self.population)//2])
                parent2 = random.choice(self.population[:len(self.population)//2])

                # Crossover
                child1, child2 = crossover_genomes(parent1, parent2, self.config)

                # Mutation
                child1 = mutate_genome(child1, self.config)
                child2 = mutate_genome(child2, self.config)

                new_population.extend([child1, child2])

            self.population = new_population[:self.config['population_size']]

        print("\n" + "="*60)
        print("EVOLUTION COMPLETED")
        print("="*60)

        return self.best_individual if self.best_individual else self.population[0]


print("✓ HybridNeuroevolutionImage class defined (with 5-fold sequential CV, CUDA-safe)")

✓ HybridNeuroevolutionImage class defined (with 5-fold sequential CV, CUDA-safe)


## 7. Execute Evolution and Display Results

In [ ]:
## Execute the hybrid neuroevolution
import time

print("="*80)
print("STARTING HYBRID NEUROEVOLUTION FOR IMAGE CLASSIFICATION")
print("="*80)
print(f"Dataset: {CONFIG['dataset']}")
print(f"Data path: {CONFIG['data_path']}")
print(f"Population size: {CONFIG['population_size']}")
print(f"Max generations: {CONFIG['max_generations']}")
print(f"Fitness threshold: {CONFIG['fitness_threshold']}%")
print(f"Image size: {CONFIG['image_width']}x{CONFIG['image_height']}")
print(f"Batch size: {CONFIG['batch_size']}")
print(f"Using 5-fold cross-validation (sequential, CUDA-safe)")
print("="*80)

# Create neuroevolution instance
neuroevolution = HybridNeuroevolutionImage(CONFIG)

# Start evolution
start_time = time.time()
best_genome = neuroevolution.evolve()
end_time = time.time()

execution_time = end_time - start_time

print(f"\n{'='*80}")
print("EVOLUTION COMPLETED SUCCESSFULLY!")
print(f"{'='*80}")
print(f"Total execution time: {execution_time:.2f} seconds ({execution_time/60:.2f} minutes)")
print(f"Generations completed: {neuroevolution.generation}")
print(f"Best fitness achieved: {best_genome['fitness']:.2f}%")
print(f"Best individual ID: {best_genome['id']}")
print(f"{'='*80}")

STARTING HYBRID NEUROEVOLUTION FOR IMAGE CLASSIFICATION
Dataset: IMAGE
Data path: data/sets/folds_5/images_all_real_syn_n
Population size: 20
Max generations: 100
Fitness threshold: 80.0%
Image size: 64x32
Batch size: 64
Using 5-fold cross-validation (sequential, CUDA-safe)

STARTING HYBRID NEUROEVOLUTION FOR IMAGES
Initializing population of 20 individuals...
✓ Population initialized with 20 individuals

GENERATION 1/100

Evaluating 20 individuals (5-fold CV)...

Individual 1/20 - ID: b7b77b4c

   Evaluating Individual b7b77b4c...

   📐 Architecture of Individual b7b77b4c:
      Conv2D Layers: 2
         L1:  12 filters, K=3x3, Pool, BN
         L2:  24 filters, K=3x3, Pool, No-BN
      FC Layers: 1
         FC1:   96 nodes
      Activation: relu, Dropout: 0.228
      Optimizer: adam, LR: 0.0005
      Training Fold 1/5...
      Loading data for Fold 1...
         Target resize: 32x64 (HxW), Channels: 3
          Fold 1 Epoch 1/10: loss=0.6818, acc=61.80%, best=61.80%, patience=2
     

Traceback (most recent call last):
  File "/tmp/ipykernel_8936/2496447629.py", line 137, in _train_fold_in_thread
    model = EvolvableCNN2D(genome, self.config).to(device)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_8936/4096732920.py", line 19, in __init__
    self.conv_output_size = self._calculate_conv_output_size()
                            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_8936/4096732920.py", line 113, in _calculate_conv_output_size
    x = layer(x)
        ^^^^^^^^
  File "/home/jovyan/.local/lib/python3.11/site-packages/torch/nn/modules/module.py", line 1773, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/jovyan/.local/lib/python3.11/site-packages/torch/nn/modules/module.py", line 1784, in _call_impl
    return forward_call(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/jovyan/.local/lib/python3.11/site-packages/torch/nn/mod

      ERROR in Fold 4: Given input size: (128x1x2). Calculated output size: (128x0x1). Output size is too small
      → Fold 4 completed: 0.00%
      Training Fold 5/5...
      Loading data for Fold 5...
         Target resize: 32x64 (HxW), Channels: 3
      ERROR in Fold 5: Given input size: (128x1x2). Calculated output size: (128x0x1). Output size is too small
      → Fold 5 completed: 0.00%
   → Individual 48d52b4d: Average Fitness = 0.00%
      Fold details: {1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0}

Individual 7/20 - ID: 596cd63e

   Evaluating Individual 596cd63e...

   📐 Architecture of Individual 596cd63e:
      Conv2D Layers: 5
         L1:  10 filters, K=7x7, No-Pool, BN
         L2:  20 filters, K=1x1, Pool, No-BN
         L3:  40 filters, K=5x5, Pool, No-BN
         L4:  80 filters, K=3x3, No-Pool, BN
         L5: 128 filters, K=1x1, Pool, BN
      FC Layers: 7
         FC1: 1015 nodes
         FC2:  507 nodes
         FC3:  253 nodes
         FC4:  126 nodes
         FC5:  

In [ ]:
# Display best architecture
def display_best_architecture(genome: dict, config: dict):
    """Display detailed information about the best architecture found."""
    print(f"\n{'='*80}")
    print("BEST ARCHITECTURE DETAILS")
    print(f"{'='*80}")
    
    print(f"\nGenome ID: {genome['id']}")
    print(f"Fitness (5-fold CV average): {genome['fitness']:.2f}%")
    
    print(f"\n--- ARCHITECTURE ---")
    print(f"Convolutional Layers (2D): {genome['num_conv_layers']}")
    for i in range(genome['num_conv_layers']):
        print(f"  Layer {i+1}: {genome['filters'][i]} filters, kernel={genome['kernel_sizes'][i]}x{genome['kernel_sizes'][i]}, " +
              f"pool={'Yes' if genome['use_pooling'][i] else 'No'}, " +
              f"bn={'Yes' if genome['use_batch_norm'][i] else 'No'}")
    
    print(f"\nFully Connected Layers: {genome['num_fc_layers']}")
    for i in range(genome['num_fc_layers']):
        print(f"  FC Layer {i+1}: {genome['fc_nodes'][i]} nodes")
    
    print(f"\n--- HYPERPARAMETERS ---")
    print(f"Activation: {genome['activation']}")
    print(f"Dropout rate: {genome['dropout_rate']:.3f}")
    print(f"Optimizer: {genome['optimizer']}")
    print(f"Learning rate: {genome['learning_rate']}")
    
    print(f"\n--- DATASET INFO ---")
    print(f"Dataset: Images (Spectrograms)")
    print(f"Image size: {config['image_width']}x{config['image_height']}")
    print(f"Channels: {config['num_channels']} (RGB)")
    print(f"Classes: {config['num_classes']} (Control vs Pathological)")
    print(f"Batch size: {config['batch_size']}")
    
    print(f"\n--- PERFORMANCE ---")
    if genome['fitness'] >= config['fitness_threshold']:
        print(f"✓ TARGET REACHED: {genome['fitness']:.2f}% >= {config['fitness_threshold']}%")
    else:
        print(f"✗ TARGET NOT REACHED: {genome['fitness']:.2f}% < {config['fitness_threshold']}%")
        print(f"  Gap: {config['fitness_threshold'] - genome['fitness']:.2f}%")
    
    # Save results to JSON
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    results_file = f"best_architecture_images_{timestamp}.json"
    
    results_data = {
        'timestamp': timestamp,
        'dataset_type': 'image_2D',
        'dataset_id': config.get('dataset_id', 'N/A'),
        'best_genome': genome,
        'final_generation': neuroevolution.generation,
        'evolution_stats': neuroevolution.generation_stats,
        'config': {k: v for k, v in config.items() if k not in ['normalization']}
    }
    
    try:
        with open(results_file, 'w') as f:
            json.dump(results_data, f, indent=2, default=str)
        print(f"\n✓ Results saved to: {results_file}")
    except Exception as e:
        print(f"\n✗ WARNING: Error saving results: {e}")
    
    print(f"\n{'='*80}")
    print("HYBRID NEUROEVOLUTION FOR IMAGES COMPLETED!")
    print(f"{'='*80}")

# Display the best architecture
display_best_architecture(best_genome, CONFIG)

In [ ]:
# Visualize evolution progress
plt.figure(figsize=(14, 6))

# Plot 1: Fitness over generations
plt.subplot(1, 2, 1)
generations = [stat['generation'] for stat in neuroevolution.generation_stats]
best_fitnesses = [stat['best_fitness'] for stat in neuroevolution.generation_stats]
avg_fitnesses = [stat['avg_fitness'] for stat in neuroevolution.generation_stats]

plt.plot(generations, best_fitnesses, 'b-o', label='Best Fitness', linewidth=2, markersize=6)
plt.plot(generations, avg_fitnesses, 'r--s', label='Average Fitness', linewidth=2, markersize=4)
plt.axhline(y=CONFIG['fitness_threshold'], color='g', linestyle=':', label=f'Target ({CONFIG["fitness_threshold"]}%)')
plt.xlabel('Generation', fontsize=12)
plt.ylabel('Fitness (%)', fontsize=12)
plt.title('Evolution Progress - Image Classification', fontsize=14, fontweight='bold')
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)

# Plot 2: Architecture complexity over generations
plt.subplot(1, 2, 2)
# Get architecture info from generation stats
conv_layers = []
fc_layers = []

for gen in generations:
    # Find best individual of this generation
    gen_stat = neuroevolution.generation_stats[gen-1]
    best_id = gen_stat['best_id']
    
    # Find genome (from population or best)
    if neuroevolution.best_individual and neuroevolution.best_individual['id'] == best_id:
        genome = neuroevolution.best_individual
    else:
        # Try to find in current population
        genome = next((ind for ind in neuroevolution.population if ind['id'] == best_id), None)
        if not genome:
            # Use current best as fallback
            genome = neuroevolution.best_individual if neuroevolution.best_individual else neuroevolution.population[0]
    
    conv_layers.append(genome['num_conv_layers'])
    fc_layers.append(genome['num_fc_layers'])

plt.plot(generations, conv_layers, 'b-o', label='Conv2D Layers', linewidth=2, markersize=6)
plt.plot(generations, fc_layers, 'r--s', label='FC Layers', linewidth=2, markersize=4)
plt.xlabel('Generation', fontsize=12)
plt.ylabel('Number of Layers', fontsize=12)
plt.title('Architecture Complexity Evolution', fontsize=14, fontweight='bold')
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)

plt.tight_layout()
# Build filename using an f-string and single quotes inside strftime to avoid nested quote conflicts
filename = f"evolution_progress_images_{datetime.now().strftime('%Y%m%d_%H%M%S')}.png"
plt.savefig(filename, dpi=150, bbox_inches='tight')
plt.show()